In [17]:
import kfp, os
import kfp.dsl as dsl 
import kfp.compiler as compiler
import kubernetes.client.models as k8s
import namesgenerator

In [64]:
NAMESPACE = os.environ.get("NAMESPACE")

In [65]:
kubeflow_address = f"http://{NAMESPACE}.kubeflow.odsc.k8s.hydrosphere.io"
hydrosphere_address = f"http://{NAMESPACE}.serving.odsc.k8s.hydrosphere.io"

In [47]:
!docker login

Authenticating with existing credentials...
Login Succeeded


# Download

At this stage we will obtain all training data for our pipeline.

Build & publish an image

In [21]:
%%bash 
docker build -t tidylobster/mnist-pipeline-download:latest --no-cache 01_download
docker push tidylobster/mnist-pipeline-download:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./download.py /src/
 ---> 087ee3703581
Step 3/4 : WORKDIR /src/
 ---> Running in d14ae15c6527
Removing intermediate container d14ae15c6527
 ---> dca52dfba5df
Step 4/4 : ENTRYPOINT [ "python", "download.py" ]
 ---> Running in 79e79db64d27
Removing intermediate container 79e79db64d27
 ---> 437844974e43
Successfully built 437844974e43
Successfully tagged tidylobster/mnist-pipeline-download:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-download]
64834980591b: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
93ed1238773c: Waiting
eeb5ce6b3db4: Waiting
886601877ba4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd

### Pipeline

Create Kubernetes PVC resource

In [22]:
storage_pvc = k8s.V1PersistentVolumeClaimVolumeSource(claim_name="storage")
storage_volume = k8s.V1Volume(name="storage", persistent_volume_claim=storage_pvc)
storage_volume_mount = k8s.V1VolumeMount(mount_path="{{workflow.parameters.mount-path}}", name="storage")

Create required environmnet variables 

In [23]:
mount_path_env = k8s.V1EnvVar(name="MOUNT_PATH", value="{{workflow.parameters.mount-path}}")

Define container operation

In [24]:
username = "tidylobster"

In [25]:
def download_op():
    download = dsl.ContainerOp(
        name="download",
        image=f"{username}/mnist-pipeline-download:latest",  # <-- Replace with built docker image
        file_outputs={"data_path": "/data_path.txt"})
    
    download.add_volume(storage_volume)
    download.add_volume_mount(storage_volume_mount)
    download.add_env_variable(mount_path_env)
    
    return download

In [26]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
):
    download = download_op()

In [27]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [28]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml

x pipeline.yaml


### Test

In [66]:
# Create Pipelines client
client = kfp.Client(kubeflow_address)

In [67]:
# Define an experiment name
experiment_name='MNIST Showreal'

In [68]:
# get or create an experiment_id
try:
    experiment_id = client.get_experiment(experiment_name=experiment_name).id
except:
    experiment_id = client.create_experiment(experiment_name).id

In [69]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(experiment_id, run_name, "pipeline.yaml")

Starting a new run with the name amazing_davinci


# Train

At this stage we will create a model & train it on the downloaded data.

Build and publish image

In [15]:
%%bash 
docker build -t tidylobster/mnist-pipeline-train:latest --no-cache 02_train/ 
docker push tidylobster/mnist-pipeline-train:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./train.py /src/
 ---> 6d6d3ee05d63
Step 3/4 : WORKDIR /src/
 ---> Running in 04dc3e9b4a8d
Removing intermediate container 04dc3e9b4a8d
 ---> 3f1cf427c4d5
Step 4/4 : ENTRYPOINT [ "python", "train.py" ]
 ---> Running in b6887fdfdfce
Removing intermediate container b6887fdfdfce
 ---> a5a20bfbdf98
Successfully built a5a20bfbdf98
Successfully tagged tidylobster/mnist-pipeline-train:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-train]
2f081c0fe8f5: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
93ed1238773c: Waiting
eeb5ce6b3db4: Waiting
886601877ba4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7: Wa

### Pipeline

Create required environmnet variables

In [10]:
learning_rate_env = k8s.V1EnvVar(name="LEARNING_RATE", value="{{workflow.parameters.learning-rate}}")
epochs_env = k8s.V1EnvVar(name="EPOCHS", value="{{workflow.parameters.epochs}}")
batch_size_env = k8s.V1EnvVar(name="BATCH_SIZE", value="{{workflow.parameters.batch-size}}")

Define container operation

In [11]:
def train_op(download):
    train = dsl.ContainerOp(
        name="train",
        image=f"{username}/mnist-pipeline-train:latest",        # <-- Replace with correct docker image
        file_outputs={"accuracy": "/accuracy.txt"},
        arguments=[download.outputs["data_path"]])

    train.add_volume(storage_volume)
    train.add_volume_mount(storage_volume_mount)
    train.add_env_variable(mount_path_env)
    train.add_env_variable(learning_rate_env)
    train.add_env_variable(epochs_env)
    train.add_env_variable(batch_size_env)
    
    return train

In [18]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
):
    
    download = download_op()
    train = train_op(download)
    
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')

In [19]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [20]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml

x pipeline.yaml


### Test

In [21]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "1"
    }
)

Starting a new run with the name optimistic_mclean


# Upload 

At this stage we will upload the model to Hydrosphere

Build and publish image

In [22]:
%%bash 
docker build -t tidylobster/mnist-pipeline-upload:latest --no-cache 03_upload/ 
docker push tidylobster/mnist-pipeline-upload:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./execute.sh /src/
 ---> 2f8f5a03a44b
Step 3/4 : WORKDIR /src/
 ---> Running in 06adbf330385
Removing intermediate container 06adbf330385
 ---> c5bd192ed610
Step 4/4 : ENTRYPOINT [ "bash", "execute.sh" ]
 ---> Running in 33b22a212bd0
Removing intermediate container 33b22a212bd0
 ---> 8e1bd7aed8cc
Successfully built 8e1bd7aed8cc
Successfully tagged tidylobster/mnist-pipeline-upload:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-upload]
5b9ce8d64610: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
93ed1238773c: Waiting
eeb5ce6b3db4: Waiting
886601877ba4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
0de2edf7bff4: Waiting
f67191ae09b8

### Pipeline

Create required environmnet variables

In [12]:
model_name_env = k8s.V1EnvVar(name="MODEL_NAME", value="{{workflow.parameters.model-name}}")
hydrosphere_address_env = k8s.V1EnvVar(name="CLUSTER_ADDRESS", value="{{workflow.parameters.hydrosphere-address}}")

Define container operation

In [13]:
def upload_op(download, train):
    upload = dsl.ContainerOp(
        name="upload",
        image=f"{username}/mnist-pipeline-upload:latest",        # <-- Replace with correct docker image
        file_outputs={"model-version": "/model-version.txt"},
        arguments=[train.outputs["accuracy"], download.outputs["data_path"]])

    upload.add_volume(storage_volume) 
    upload.add_volume_mount(storage_volume_mount)
    upload.add_env_variable(mount_path_env)
    upload.add_env_variable(model_name_env)
    upload.add_env_variable(hydrosphere_address_env)
    upload.add_env_variable(learning_rate_env)
    upload.add_env_variable(epochs_env)
    upload.add_env_variable(batch_size_env)
    
    return upload

In [25]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address=""
):
    
    download = download_op()
    
    train = train_op(download)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    upload = upload_op(download, train)
    upload.after(train)

In [26]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [27]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml

x pipeline.yaml


### Test

In [28]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
    }
)

Starting a new run with the name elastic_fermi


# Predeploy

At this stage we are pre-deploying the application to run integration tests.

Build and publish image

In [29]:
%%bash 
docker build -t tidylobster/mnist-pipeline-predeploy:latest --no-cache 04_predeploy/ 
docker push tidylobster/mnist-pipeline-predeploy:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./execute.sh /src/
 ---> 986b888f1837
Step 3/4 : WORKDIR /src/
 ---> Running in adcd32c12c39
Removing intermediate container adcd32c12c39
 ---> 43d9b7d1efeb
Step 4/4 : ENTRYPOINT [ "bash", "execute.sh" ]
 ---> Running in 3e917293df41
Removing intermediate container 3e917293df41
 ---> 9080334b7286
Successfully built 9080334b7286
Successfully tagged tidylobster/mnist-pipeline-predeploy:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-predeploy]
0a2f84caf498: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
93ed1238773c: Waiting
eeb5ce6b3db4: Waiting
886601877ba4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd8b

### Pipeline

Create required environmnet variables

In [14]:
application_name_env = k8s.V1EnvVar(name="APPLICATION_NAME", value="{{workflow.parameters.application-name}}")

Define container operation

In [15]:
def predeploy_op(upload):
    predeploy = dsl.ContainerOp(
        name="predeploy",
        image=f"{username}/mnist-pipeline-predeploy:latest",        # <-- Replace with correct docker image
        arguments=[upload.outputs["model-version"]],
        file_outputs={"predeploy-app-name": "/predeploy-app-name.txt"})

    predeploy.add_env_variable(hydrosphere_address_env)
    predeploy.add_env_variable(application_name_env)
    predeploy.add_env_variable(model_name_env)
    
    return predeploy

In [32]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    application_name="mnist-app"
):
    
    download = download_op()
    
    train = train_op(download)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    upload = upload_op(download, train)
    upload.after(train)
    
    predeploy = predeploy_op(upload)
    predeploy.after(upload)


In [33]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [34]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml

x pipeline.yaml


### Test

In [35]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "application-name": "mnist-app",
    }
)

Starting a new run with the name optimistic_leavitt


# Test

At this stage we are performing integration tests.

Build and publish image

In [36]:
%%bash 
docker build -t tidylobster/mnist-pipeline-test:latest --no-cache 05_test/ 
docker push tidylobster/mnist-pipeline-test:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./test.py /src/
 ---> 045e1ae7d4d4
Step 3/4 : WORKDIR /src/
 ---> Running in 1f9ca431d3d6
Removing intermediate container 1f9ca431d3d6
 ---> ac29e11563c5
Step 4/4 : ENTRYPOINT [ "python", "test.py" ]
 ---> Running in dddc68c4da11
Removing intermediate container dddc68c4da11
 ---> dda556327cc5
Successfully built dda556327cc5
Successfully tagged tidylobster/mnist-pipeline-test:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-test]
61de64499d97: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
886601877ba4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7: Waiting
0de2edf7bff4: Waiting
eeb5ce6b3db4: Waitin

### Pipeline

Create required environmnet variables

In [16]:
test_amount_env = k8s.V1EnvVar(name="TEST_AMOUNT", value="{{workflow.parameters.test-amount}}")
requests_delay_env = k8s.V1EnvVar(name="REQUESTS_DELAY", value="{{workflow.parameters.requests-delay}}")
acceptable_accuracy_env = k8s.V1EnvVar(name="ACCEPTABLE_ACCURACY", value="{{workflow.parameters.acceptable-accuracy}}")

Define container operation

In [17]:
def test_op(download, predeploy):
    test = dsl.ContainerOp(
        name="test",
        image=f"{username}/mnist-pipeline-test:latest",               # <-- Replace with correct docker image
        arguments=[predeploy.outputs["predeploy-app-name"], download.outputs["data_path"]])

    test.add_volume(storage_volume) 
    test.add_volume_mount(storage_volume_mount)
    test.add_env_variable(mount_path_env)
    test.add_env_variable(hydrosphere_address_env)
    test.add_env_variable(application_name_env)
    test.add_env_variable(test_amount_env)
    test.add_env_variable(acceptable_accuracy_env)
    test.add_env_variable(requests_delay_env)
    
    return test

In [39]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    application_name="mnist-app",
    acceptable_accuracy="0.90",
    test_amount="100",
    requests_delay="2",
):
    
    download = download_op()
    
    train = train_op(download)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    upload = upload_op(download, train)
    upload.after(train)
    
    predeploy = predeploy_op(upload)
    predeploy.after(upload)
    
    test = test_op(download, predeploy)
    test.set_retry(3)
    test.after(predeploy)

In [40]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [41]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml

x pipeline.yaml


### Test

In [42]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "application-name": "mnist-app",
        "test-amount": "100",
        "requests-delay": "2",
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name silly_varahamihira


# Deploy

And finally deploying application to production

Build and publish image

In [60]:
%%bash 
docker build -t tidylobster/mnist-pipeline-deploy:latest --no-cache 06_deploy/ 
docker push tidylobster/mnist-pipeline-deploy:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./execute.sh /src/
 ---> da76605d12f7
Step 3/4 : WORKDIR /src/
 ---> Running in 17dcf47b6f23
Removing intermediate container 17dcf47b6f23
 ---> 537dcafe6a73
Step 4/4 : ENTRYPOINT [ "bash", "execute.sh" ]
 ---> Running in a647443ae8d6
Removing intermediate container a647443ae8d6
 ---> ca5c8547e0d8
Successfully built ca5c8547e0d8
Successfully tagged tidylobster/mnist-pipeline-deploy:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-deploy]
722a60b5129a: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
886601877ba4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7: Waiting
93ed1238773c: Waiting
eeb5ce6b3db4

### Pipeline

Define container operation

In [18]:
def deploy_op(upload):
    deploy = dsl.ContainerOp(
        name="deploy",
        image=f"{username}/mnist-pipeline-deploy:latest",              # <-- Replace with correct docker image
        arguments=[upload.outputs["model-version"]])

    deploy.add_env_variable(hydrosphere_address_env)
    deploy.add_env_variable(application_name_env)
    deploy.add_env_variable(model_name_env)
    
    return deploy

In [19]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    application_name="mnist-app",
    acceptable_accuracy="0.90",
    test_amount="100",
    requests_delay="2",
):
    
    download = download_op()
    
    train = train_op(download)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    upload = upload_op(download, train)
    upload.after(train)
    
    predeploy = predeploy_op(upload)
    predeploy.after(upload)
    
    test = test_op(download, predeploy)
    test.set_retry(3)
    test.after(predeploy)
    
    deploy = deploy_op(upload)
    deploy.after(test)
    

In [20]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [21]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml

x pipeline.yaml


### Test

In [22]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "application-name": "mnist-app",
        "test-amount": "100",
        "requests-delay": "2",
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name vigorous_albattani


# Sample

Once we have a fully functional pipeline, we would like to automate running this pipeline. But we don't 

Build and publish image

In [49]:
%%bash 
docker build -t tidylobster/mnist-pipeline-sample:latest --no-cache 01_sample/ 
docker push tidylobster/mnist-pipeline-sample:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./sample.py /src/
 ---> bfb058e25199
Step 3/4 : WORKDIR /src/
 ---> Running in 29e23b657ee3
Removing intermediate container 29e23b657ee3
 ---> 2f4747efea6a
Step 4/4 : ENTRYPOINT [ "python", "sample.py" ]
 ---> Running in bbfe33fe0d33
Removing intermediate container bbfe33fe0d33
 ---> 3a2a04fe2c08
Successfully built 3a2a04fe2c08
Successfully tagged tidylobster/mnist-pipeline-sample:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-sample]
5b283f22ec6c: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
886601877ba4: Waiting
93ed1238773c: Waiting
eeb5ce6b3db4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7

### Pipeline

Define container operation

In [50]:
def sample_op():
    sample = dsl.ContainerOp(
        name="sample",
        image=f"{username}/mnist-pipeline-sample:latest", # <-- Replace with correct docker image
        file_outputs={"data_path": "/data_path.txt"})  
    
    sample.add_volume(storage_volume)
    sample.add_volume_mount(storage_volume_mount)
    sample.add_env_variable(mount_path_env)
    sample.add_env_variable(hydrosphere_address_env)
    sample.add_env_variable(application_name_env)
    
    return sample

In [51]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    application_name="mnist-app",
    acceptable_accuracy="0.90",
    test_amount="100",
    requests_delay="2",
):
    
    sample = sample_op()
    
    train = train_op(sample)
    train.after(sample)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    upload = upload_op(sample, train)
    upload.after(train)
    
    predeploy = predeploy_op(upload)
    predeploy.after(upload)
    
    test = test_op(sample, predeploy)
    test.set_retry(3)
    test.after(predeploy)
    
    deploy = deploy_op(upload)
    deploy.after(test)

In [52]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [53]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml

x pipeline.yaml


### Test

In [54]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "application-name": "mnist-app",
        "test-amount": "100",
        "requests-delay": "2",
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name focused_spence
